In [50]:
from tkinter import *
import numpy as np
import copy


In [55]:
#Coordinates go from 0 to 8
#Numbers go from 1 to 9, 0 means empty cell
#"pm": Pencilmark

#Number of supported 1-cell shapes (including no-shape)
#0 Nothing
#1 Filled circle
#2 Empty circle
#3 Filled square
#4 Empty square
#5 Min cell
#6 Max cell   
#7 Encircled cell (for summing arrows)
SHAPE1NR = 8 

#Number of supported 2-cell shapes (including no-shape)
#0 Nothing
#1 Black Kropki dot
#2 White Kropki dot
#3 Black Kropki square
#4 White Kropki square
#5 X
#6 V
#7 <
#8 >
#9 Encircled double cell (for summing arrows)
SHAPE2NR = 10

#Number of line segment types (including no-shape)
#0 Nothing
#1 Thermo line
#2 Thin line
#3 Thin line with arrow at the beginning
#4 Thin line with arrow at the end
LINESNR = 5


class Grid:
    def __init__(self, orig = None):
        self.grid       = np.zeros( (9,9) ).astype(np.int8)
        
        #boolean for indicating digits belonging to the puzzle (and not the solution)
        self.orig   = np.zeros( (9,9) ).astype(np.int8) 
        
        #this is handled by the class, though user can change it
        self.options    = np.zeros( (9,9,10) ).astype(np.int8) #boolean for each option (i.e. centerpm)
        self.options_nr = np.zeros( (9,9) ).astype(np.int8)    #nr of options
        self.options_ch = np.zeros( (9,9) ).astype(np.int8)    #boolean if options were changed (by hand) in a cell 
        self.options_sh = np.zeros( (9,9) ).astype(np.int8)    #boolean if options are shown in a cell 

        #this is handled by the user
        self.toppm      = np.zeros( (9,9,10) ).astype(np.int8) #boolean for each option
        
        #this is handled by the user
        self.color      = np.zeros( (9,9) ).astype(np.int8)    #coloring of the cells; 0 -> no coloring
        
        #Showing options automatically if their number <= self.showlevel
        self.showlevel = 3
        
        #1-cell shapes: List of (type, i, j); Types:
        self.shape1 = []

        #2-cell shapes: List of (type, i1, j1, i2, j2); Types:
        self.shape2 = []
        
        #Line segments behave as 2-cell shapes: (type, i1, j1, i2, j2)
        self.lines = []
        
        self.init_grid()
        
    def init_grid(self):
        for i in range(9):
            for j in range(9):
                self.grid[i,j]       = 0    #The cell is empty at the beginning
                self.orig[i,j]       = 0    #The grid is empty at the beginning (no given digits)
                self.options_nr[i,j] = 9    #All options are available at the beginning -> #(1..9)=9
                self.options_sh[i,j] = 0    #Not showing the options at the beginning
                self.options_ch[i,j] = 0    #No change at the beginning 
                self.options[i,j,0]  = 0    #0 is never an option
                self.toppm[i,j,0]    = 0    #0 is never a top pm
                self.color[i,j]      = 0    #No coloring for each cell
                for k in range(1,10):
                    self.options[i,j,k] = 1 #All options are available at the beginning
                    self.toppm[i,j,k]   = 0 #There is no top pm at the beginning
                    
    def get_digit(self,i,j):
        return self.grid[i,j]
    
    def is_orig(self,i,j):
        return self.orig[i,j]

    def get_centerpm(self,i,j,forced=False):
        res = ''
        if self.options_sh[i,j] or forced:
            for k in range(1,10):
                if self.options[i,j,k]==1:
                    res += str(k)
        return res

    def is_centerpm(self,i,j,digit):
        return self.options[i,j,digit]

    def get_toppm(self,i,j):
        res = ''
        for k in range(1,10):
            if self.toppm[i,j,k]==1:
                res += str(k)
        return res

    def is_toppm(self,i,j,digit):
        return self.toppm[i,j,digit]
    
    
    def get_color(self, i, j):
        return self.color[i,j]

    def get_shape1(self):
        return self.shape1

    def get_shape2(self):
        return self.shape2

    def get_lines(self):
        return self.lines
    
    def add_digit(self, i, j, digit, orig=False):
        if self.options[i,j,digit]==0:
            return False
        self.grid[i,j]=digit
        if orig:
            self.orig[i,j]=1
        self.update_options()
        self.update_show()
        return True
    
    def change_centerpm(self,i,j,digit):
        #If not visible -> no change
        if not self.options_sh[i,j]:
            return False
        if self.options[i,j,digit]==1:
            self.options[i,j,digit] = 0
            self.options_nr[i,j] -= 1
            self.options_ch[i,j] = 1
            self.toppm[i,j,digit]=0
            return True
        else:
            return False
            
    def is_centerpm_shown(self,i,j):
        return self.options_sh[i,j]
            
    def toggle_centerpm_shown(self,i,j):        
        if self.options_sh[i,j] and self.options_ch[i,j]:
            return False
        self.options_sh[i,j] = 1 - self.options_sh[i,j]           
        return True
            
    def change_toppm(self,i,j,digit):
        if self.toppm[i,j,digit]==1:
            self.toppm[i,j,digit] = 0
            return True
        else:
            valid=self.options[i,j,digit]
            if valid:
                self.toppm[i,j,digit]=1
            return valid
    
    def change_color(self,i,j,color):
        res = False
        if self.color[i,j] != color:
            self.color[i,j] = color
            res = True
        return res
    
    def update_options(self):
        for i in range(9):
            for j in range(9):
                if self.grid[i,j]!=0:                    
                    d = self.grid[i,j]
                    for dd in range(1,10):
                        self.toppm[i,j,dd]=0
                        if d!=dd:
                            if self.options[i,j,dd]==1:
                                self.options[i,j,dd]=0                        
                                self.options_nr[i,j]-=1                            
                    for k in range(9):
                        self.toppm[i,k,d]=0
                        self.toppm[k,j,d]=0
                        if k!=j:
                            if self.options[i,k,d] == 1:
                                self.options[i,k,d] = 0
                                self.options_nr[i,k] -= 1                                            
                        if k!=i:
                            if self.options[k,j,d] == 1:
                                self.options[k,j,d] = 0
                                self.options_nr[k,j] -= 1
                    i0 = i - i%3
                    j0 = j - j%3
                    for ii in range(3):
                        for jj in range(3):
                            self.toppm[i0+ii,j0+jj,d]=0
                            if i0+ii!=i or j0+jj!=j:
                                if self.options[i0+ii,j0+jj,d]==1:
                                    self.options[i0+ii,j0+jj,d]=0
                                    self.options_nr[i0+ii,j0+jj]-=1
    
    def update_show(self):
        for i in range(9):
            for j in range(9):
                if self.options_nr[i,j] <= self.showlevel:
                    self.options_sh[i,j] = 1

    def is_broken(self):
        res = False
        for i in range(9):
            for j in range(9):
                if self.options_nr[i,j]==0:
                    res = True
        return res

    def is_solved(self):
        res = True
        for i in range(9):
            for j in range(9):
                if self.grid[i,j]==0:
                    res = False
        return res
    
    def change_showlevel(self, delta):
        res = False
        if 0<=self.showlevel+delta<=9:
            self.showlevel += delta
            for i in range(9):
                for j in range(9):
                    if self.options_sh[i,j] and \
                       not self.options_ch[i,j] and \
                       self.options_nr[i,j] > self.showlevel:
                        self.options_sh[i,j]=0
                    elif not self.options_sh[i,j] and \
                         self.options_nr[i,j] <= self.showlevel:
                        self.options_sh[i,j]=1
            res = True
        return res

    def next_1cellshape_digit(self,i,j):
        ind = -1
        for k,s in enumerate(self.shape1):
            if s[1]==i and s[2]==j:
                ind = k
                break
        if ind==-1:
            self.shape1.append( [1,i,j] )
        else:
            self.shape1[ind][0] = (self.shape1[ind][0] + 1) % SHAPE1NR
        return True

    def next_2cellshape_digit(self,i1,j1,i2,j2):
        #Cells are ordered, only orthogonally adjacent cells are allowed
        if i1>i2 or j1>j2 or (j2-j1+i2-i1)!=1: 
            return False
        ind = -1        
        for k,s in enumerate(self.shape2):
            if s[1]==i1 and s[2]==j1 and s[3]==i2 and s[4]==j2:
                ind = k
                break
        if ind==-1:
            self.shape2.append( [1,i1,j1,i2,j2] )
        else:
            self.shape2[ind][0] = (self.shape2[ind][0] + 1) % SHAPE2NR
        return True

    def next_line_segment(self,i1,j1,i2,j2):
        #Cells are ordered, only 8-way adjacent cells are allowed
        if i1>i2 or (i1==i2 and j1>j2) or (i2-i1)>1 or abs(j2-j1)>1:
            return False
        ind = -1        
        for k,s in enumerate(self.lines):
            if s[1]==i1 and s[2]==j1 and s[3]==i2 and s[4]==j2:
                ind = k
                break
        if ind==-1:
            self.lines.append( [1,i1,j1,i2,j2] )
        else:
            self.lines[ind][0] = (self.lines[ind][0] + 1) % LINESNR
        return True
    
    

In [52]:
class GridList:
    def __init__(self):    
        grid = Grid()
        
        #List of grids, always pointing to the current list in self.listlist
        #This is the undo-redo list for solving the sudoku from one particular starting position
        self.gridlist = [grid]

        #List of girdlists for the different starting positions (including the current pointer [curri])
        #This is the undo-redo list for the different starting positions 
        #including the gridlists belonging to them
        self.listlist = [ [self.gridlist,0] ]
        
        self.listi = 0 #Index of current list
        self.curri = 0 #Index of current grid in the current list

    def appendgrid(self,newgrid):
        del self.gridlist[self.curri+1:]
        self.gridlist.append(newgrid)
        self.curri = len(self.gridlist)-1
        
        
    def getcurr(self):
        res =  self.gridlist[self.curri]
        return res 
        
    def add_digit(self, selected, i, j, digit):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = True
        if selected:
            for (ii,jj) in selected:                
                if not newgrid.add_digit(ii,jj,digit):
                    success = False
                    break
        else:
            success = newgrid.add_digit(i,j,digit)
        if success:
            self.appendgrid(newgrid)

    def change_centerpm(self, selected, i, j, digit):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected: 
            nr = 0
            for (ii,jj) in selected:
                if newgrid.is_centerpm(ii,jj,digit):
                    nr += 1
            for (ii,jj) in selected:
                if not newgrid.is_centerpm_shown(ii,jj):
                    success = False # No change if any CenterPM is invisible in the selection
                    break
                    #newgrid.toggle_centerpm_shown(ii,jj)
                if nr>0:
                    if newgrid.is_centerpm(ii,jj,digit):
                        success |= newgrid.change_centerpm(ii,jj,digit)                                    
                else:
                    if not newgrid.is_centerpm(ii,jj,digit):
                        success |= newgrid.change_centerpm(ii,jj,digit)                    
        else: #no selection: using currently framed cell
            success = newgrid.change_centerpm(i,j,digit)
        if success:
            self.appendgrid(newgrid)

    def change_toppm(self, selected, i, j, digit):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected: 
            nr=0
            for (ii,jj) in selected:
                if newgrid.is_toppm(ii,jj,digit):
                    nr += 1
            for (ii,jj) in selected:
                if nr>0:
                    if newgrid.is_toppm(ii,jj,digit):
                        success |= newgrid.change_toppm(ii,jj,digit)                    
                else:
                    if not newgrid.is_toppm(ii,jj,digit):
                        success |= newgrid.change_toppm(ii,jj,digit)                    
        else:
            success = newgrid.change_toppm(i,j,digit)
        if success:
            self.appendgrid(newgrid)
            
    def change_color(self, selected, i, j, color):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected:             
            for (ii,jj) in selected:
                success |= newgrid.change_color(ii,jj,color)
        else:
            success = newgrid.change_color(i,j,color)
        if success:
            self.appendgrid(newgrid)        
           
    def centerpm_onoff(self, selected, i, j):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected: 
            #Success: If at least one cell was changed
            #If not all shown -> 
            #  1: show all
            #  2: then delete those which can be deleted
            nr=0
            for (ii,jj) in selected:
                if not newgrid.is_centerpm_shown(ii,jj):
                    nr += 1
            for (ii,jj) in selected:
                if nr>0:
                    if not newgrid.is_centerpm_shown(ii,jj):
                        success |= newgrid.toggle_centerpm_shown(ii,jj)
                else:
                    if newgrid.is_centerpm_shown(ii,jj):
                        success |= newgrid.toggle_centerpm_shown(ii,jj)
        else:
            success = newgrid.toggle_centerpm_shown(i,j)
        if success:
            self.appendgrid(newgrid)
            
    def undo(self):
        if self.curri>0:
            self.curri -= 1
    
    def redo(self):
        if self.curri<len(self.gridlist)-1:
            self.curri += 1
    
    def is_solved(self):
        return self.gridlist[self.curri].is_solved()
        
    def is_broken(self):
        return self.gridlist[self.curri].is_broken()

    def change_showlevel(self,delta):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = newgrid.change_showlevel(delta)
        if success:
            self.appendgrid(newgrid)    
    
    def appendlist(self, newlist):
        del self.listlist[self.listi+1:]
        self.listlist[self.listi][1] = self.curri #Saving current self.curri
        self.listlist.append( [newlist,self.curri] )
        self.gridlist = newlist
        self.listi = len(self.listlist)-1                        
    
    #Adding a new digit to the starting position
    # -> This starts a new gridlist in listlist
    def add_base_digit(self, selected, i, j, digit):
        currlist = self.gridlist
        newlist  = copy.deepcopy(self.gridlist)
        del newlist[self.curri+1:]
        if selected:
            for (ii,jj) in selected:
                for grid in newlist:
                    success = grid.add_digit(ii,jj,digit,orig=True)
                    if not success:
                        break
                if not success:
                    break
        else:        
            success = True
            for grid in newlist:
                success = grid.add_digit(i,j,digit,orig=True)
                if not success:
                    break                
        if success:
            self.appendlist(newlist)
    
    def undo_base(self):
        if self.listi>0:
            self.listlist[self.listi][1] = self.curri #Saving curri
            self.listi -= 1
            self.gridlist, self.curri = self.listlist[self.listi]                    
    
    def redo_base(self):
        if self.listi<len(self.listlist)-1:
            self.listlist[self.listi][1] = self.curri #Saving curri
            self.listi += 1
            self.gridlist, self.curri = self.listlist[self.listi]
    
    def next_1cellshape_base(self, i, j):
        currlist = self.gridlist
        newlist  = copy.deepcopy(self.gridlist)
        del newlist[self.curri+1:]
        success = True
        for grid in newlist:
            success = grid.next_1cellshape_digit(i,j)
            if not success:
                break
        if success:            
            self.appendlist(newlist)
        
    def next_2cellshape_base(self, i1, j1, i2, j2):
        currlist = self.gridlist
        newlist  = copy.deepcopy(self.gridlist)
        del newlist[self.curri+1:]
        success = True
        for grid in newlist:
            success = grid.next_2cellshape_digit(i1,j1,i2,j2)
            if not success:
                break
        if success:
            self.appendlist(newlist)
            
    def next_line_segment_base(self, i1, j1, i2, j2):
        currlist = self.gridlist
        newlist  = copy.deepcopy(self.gridlist)
        del newlist[self.curri+1:]
        success = True
        for grid in newlist:
            success = grid.next_line_segment(i1,j1,i2,j2)
            if not success:
                break
        if success:
            self.appendlist(newlist)
                
            

In [74]:
VP=HP=65
VW=HW=65
TOPPM_PADDING = HP//3
PM_ADJ = 2
LW1 = 2
LW2 = 3
FRAMEWIDTH=3
SHAPEWIDTH=8
SARROW=15

FONT_L      = "Mono"
FONTSIZE_L  = "9"
FONT        = "Mono"
FONTSIZE    = "40"
FONT_PM     = "Mono"
FONTSIZE_PM = "10"
FONT_S2     = "Mono"
FONTSIZE_S2 = "12"

SOLVEDCOLOR   = 'green'
BROKENCOLOR   = 'red'
BGCOLOR       = 'white'
LINECOLOR     = 'black'
FONTCOLOR     = 'darkblue'
ORIGFONTCOLOR = 'black'
FRAMECOLOR    = 'red'
SELCOLOR      = 'yellow'
SELSTIPPLE    = 'gray50'
SHAPECOLOR    = 'gray50'

INACTIVECPMCOLOR = 'lightgray'

COLORLIST  = [BGCOLOR,'tomato','pale green',
              'deep sky blue','orange','medium orchid',
              'bisque4','cyan3','light goldenrod yellow','hot pink']
COLSTIPPLE = '' #'gray75'
COLW       = 15


class UI:
    def __init__(self):
        self.allcpm = 0 #If showing all CenterPM
        
        self.currcolor = 0 #Index of current color (from COLORLIST)
        
        self.curri = 0
        self.currj = 0
        self.selected = set()            
        
        self.framelines = []
    
        self.gridlist = GridList()

        self.root = Tk()
        self.canvas = Canvas(self.root, width=2*VP+9*VW+1, height=2*HP+9*HW+1, background=BGCOLOR)
        self.Draw(delete=False)

        self.canvas.pack()
        
        #Key bindings
        #Arrow keys
        self.canvas.bind("<Up>",    self.key_arrow)
        self.canvas.bind("<Down>",  self.key_arrow)
        self.canvas.bind("<Left>",  self.key_arrow)
        self.canvas.bind("<Right>", self.key_arrow)
        
        #Specific keys & key combinations
        self.canvas.bind("u",           self.undo)
        self.canvas.bind("r",           self.redo)
        self.canvas.bind("A",           self.select_all)
        self.canvas.bind("<Control-a>", self.deselect_all)
        self.canvas.bind("o",           self.centerpm_onoff)
        self.canvas.bind("s",           self.show_all_centerpm)
        self.canvas.bind("w",           self.inc_showlevel)
        self.canvas.bind("q",           self.dec_showlevel)
        self.canvas.bind("c",           self.set_color)
        self.canvas.bind("C",           self.inc_currcolor)
        self.canvas.bind("<Control-c>", self.dec_currcolor)
        self.canvas.bind("<Alt-u>",     self.undo_base)
        self.canvas.bind("<Alt-r>",     self.redo_base)
        self.canvas.bind("<Alt-s>",     self.next_shape_base)
        self.canvas.bind("<Alt-l>",     self.next_line_segment_base)
        
        #All other keys (including digits)
        self.canvas.bind("<Key>",   self.key)
        
        #Mouse bindings
        self.canvas.bind("<Button-1>",         self.leftclick)
        self.canvas.bind("<Shift-Button-1>",   self.leftclick)
        self.canvas.bind("<Control-Button-1>", self.leftclick)
                        
        self.canvas.focus_set()
        
    def mainloop(self):
        self.root.mainloop()        
       
    
    ##########################
    # Drawing methods
    
    def DrawBase(self, canvas):
        color = LINECOLOR
        if self.gridlist.is_solved():
            color = SOLVEDCOLOR
        elif self.gridlist.is_broken():
            color = BROKENCOLOR
        for i in range(10):
            w=LW1
            if i%3==0:
                w=LW2
            canvas.create_line(VP, HP+HW*i, VP+9*VW, HP+HW*i, fill=color, width=w)
        for i in range(10):
            w=LW1
            if i%3==0:
                w=LW2
            canvas.create_line(VP+VW*i, HP, VP+VW*i, HP+HW*9, fill=color, width=w)

        # Current color
        canvas.create_rectangle(2, 2, COLW, COLW, outline=LINECOLOR, width=1,
                                fill=COLORLIST[self.currcolor] )        
        
        #GridList info-s
        txt = "{}/{};{}/{};sh={};".format(self.gridlist.curri+1,len(self.gridlist.gridlist),
                                          self.gridlist.listi+1,len(self.gridlist.listlist),
                                          self.gridlist.gridlist[self.gridlist.curri].showlevel)
        canvas.create_text(1+VW//3+1,1,anchor = "nw",
                           fill=FONTCOLOR,
                           font=FONT_L+" "+FONTSIZE_L,
                           text=txt)        

    def DrawColors(self, canvas, grid):
        for i in range(9):
            for j in range(9):                
                t=VP+VW*i
                l=HP+HW*j
                b=VP+VW*(i+1)
                r=HP+HW*(j+1)
                colorind = grid.get_color(i,j)
                color = COLORLIST[ colorind ]
                canvas.create_rectangle(t, l, b, r, fill=color, outline='', stipple=COLSTIPPLE)        

                
    def DrawShapes(self, canvas, grid):        
        #Line segments: List of (type, i1, j1, i2, j2);
        for s in grid.get_lines():
            t0=VP+VW*s[1]+(VW+LW1)//2
            l0=HP+HW*s[2]+(HW+LW1)//2
            t1=VP+VW*s[3]+(VW+LW1)//2
            l1=HP+HW*s[4]+(HW+LW1)//2            
            if s[0]==0:   #0 No 2-cell shape -> nothing to draw
                pass
            elif s[0]==1:   #0 Thermo
                sw2 = SHAPEWIDTH//2-1
                if (s[1]!=s[3] and s[2]!=s[4]): #diagonal
                    sw2 = int(sw2/1.41)
                canvas.create_rectangle(t0-sw2-1,l0-sw2-1,t0+sw2,l0+sw2, fill=SHAPECOLOR, outline=SHAPECOLOR)
                canvas.create_rectangle(t1-sw2-1,l1-sw2-1,t1+sw2,l1+sw2, fill=SHAPECOLOR, outline=SHAPECOLOR)
                canvas.create_line(t0, l0, t1, l1, fill=SHAPECOLOR, width=SHAPEWIDTH)                
            elif s[0]==2:   #0 Thin line
                canvas.create_line(t0, l0, t1, l1, fill=SHAPECOLOR, width=LW1)                
            elif s[0]==3:   #0 Thin line with arrow at the beginning
                canvas.create_line(t0, l0, t1, l1, fill=SHAPECOLOR, width=LW1,arrow='first')                
            elif s[0]==4:   #0 Thin line with arrow at the end
                canvas.create_line(t0, l0, t1, l1, fill=SHAPECOLOR, width=LW1,arrow='last')                                                
        #2-cell shapes: List of (type, i1, j1, i2, j2); 
        for s in grid.get_shape2():
            t0=VP+VW*s[1]
            l0=HP+HW*s[2]
            if s[1]!=s[3]:
                t0 += VW
            else:
                t0 += VW//2
            if s[2]!=s[4]:
                l0 += HW
            else:
                l0 += HW//2
            t=t0-VW//8
            l=l0-HW//8
            b=t0+VW//8
            r=l0+HW//8
            
            if s[0]==0:     #No 2-cell shape -> nothing to draw
                pass
            elif s[0]==1:   #Black Kropki dot
                canvas.create_oval(t,l,b,r, fill=LINECOLOR, outline=LINECOLOR, width=LW1)
            elif s[0]==2:   #White Kropki dot
                canvas.create_oval(t,l,b,r, fill=BGCOLOR, outline=LINECOLOR, width=LW1)
            elif s[0]==3:   #Black Kropki Square
                canvas.create_rectangle(t,l,b,r, fill=LINECOLOR, outline=LINECOLOR, width=LW1)
            elif s[0]==4:   #White Kropki Square
                canvas.create_rectangle(t,l,b,r, fill=BGCOLOR, outline=LINECOLOR, width=LW1)
            elif s[0]==5:   #X
                canvas.create_oval(t,l,b,r, fill=BGCOLOR, outline='')
                canvas.create_text(t0,l0,
                   anchor = "center",
                   fill=FONTCOLOR,
                   font=FONT_S2+" "+FONTSIZE_S2+" bold",
                   text='X')
            elif s[0]==6:   #V
                canvas.create_oval(t,l,b,r, fill=BGCOLOR, outline='')
                canvas.create_text(t0,l0,
                   anchor = "center",
                   fill=FONTCOLOR,
                   font=FONT_S2+" "+FONTSIZE_S2+" bold",
                   text='V')
            elif s[0]==7:   #<
                if s[2]==s[4]:
                    s = '<'
                else:
                    s = '∨'                
                canvas.create_text(t0,l0,
                   anchor = "center",
                   fill=FONTCOLOR,
                   font=FONT_S2+" "+FONTSIZE_S2+" bold",
                   text=s)
            elif s[0]==8:   #>
                if s[2]==s[4]:
                    s = '>'
                else:
                    s = '∧'
                canvas.create_text(t0,l0,
                   anchor = "center",
                   fill=FONTCOLOR,
                   font=FONT_S2+" "+FONTSIZE_S2+" bold",
                   text=s)
            elif s[0]==9:   #Encircled double-cell for summing arrows
                t0=VP+VW*s[1]
                l0=HP+HW*s[2]
                t1=VP+VW*(s[3]+1)
                l1=HP+HW*(s[4]+1)                
                canvas.create_polygon([t0+2*LW2,l0+2*LW2,
                                       (t0+t1)//2,l0+2*LW2,
                                       t1-2*LW2,l0+2*LW2,
                                       t1-2*LW2,(l0+l1)//2,
                                       t1-2*LW2,l1-2*LW2,
                                       (t0+t1)//2,l1-2*LW2,
                                       t0+2*LW2,l1-2*LW2,
                                       t0+2*LW2,(l0+l1)//2], 
                                      outline=SHAPECOLOR, fill=BGCOLOR, width=LW1, 
                                      smooth=True)
            
        # 1-cell Shapes
        for s in grid.get_shape1():
            t0=VP+VW*s[1]
            l0=HP+HW*s[2]
            t=VP+VW*s[1]+VW//4
            l=HP+HW*s[2]+HW//4
            b=VP+VW*(s[1]+1)-VW//4
            r=HP+HW*(s[2]+1)-HW//4

            if s[0]==0:   #No 1-cell shape -> nothing to draw
                pass
            elif s[0]==1: #Filled circle
                canvas.create_oval(t,l,b,r, fill=SHAPECOLOR, outline=SHAPECOLOR, width=SHAPEWIDTH)                
            elif s[0]==2: #Empty circle
                canvas.create_oval(t,l,b,r, outline=SHAPECOLOR, width=SHAPEWIDTH)
            elif s[0]==3: #Filled square
                canvas.create_rectangle(t,l,b,r, fill=SHAPECOLOR, outline=SHAPECOLOR, width=SHAPEWIDTH)                
            elif s[0]==4: #Empty square
                canvas.create_rectangle(t,l,b,r, outline=SHAPECOLOR, width=SHAPEWIDTH)
            elif s[0]==5: #Min cell
                canvas.create_line(t0+VW//2, l0+LW1, t0+VW//2, l0+SARROW+LW1, fill=SHAPECOLOR, width=LW2, arrow='first')
                canvas.create_line(t0+VW//2, l0+HW-LW1, t0+VW//2, l0+HW-SARROW-LW1, fill=SHAPECOLOR, width=LW2, arrow='first')
                canvas.create_line(t0+LW1, l0+HW//2, t0+SARROW+LW1, l0+HW//2, fill=SHAPECOLOR, width=LW2, arrow='first')
                canvas.create_line(t0+VW-LW1, l0+HW//2, t0+VW-SARROW-LW1, l0+HW//2, fill=SHAPECOLOR, width=LW2, arrow='first')
            elif s[0]==6: #Max cell        
                canvas.create_line(t0+VW//2, l0+LW1, t0+VW//2, l0+SARROW+LW1, fill=SHAPECOLOR, width=LW2, arrow='last')
                canvas.create_line(t0+VW//2, l0+HW-LW1, t0+VW//2, l0+HW-SARROW-LW1, fill=SHAPECOLOR, width=LW2, arrow='last')
                canvas.create_line(t0+LW1, l0+HW//2, t0+SARROW+LW1, l0+HW//2, fill=SHAPECOLOR, width=LW2, arrow='last')
                canvas.create_line(t0+VW-LW1, l0+HW//2, t0+VW-SARROW-LW1, l0+HW//2, fill=SHAPECOLOR, width=LW2, arrow='last')
            elif s[0]==7: #Encricled cell for summing arrow
                canvas.create_polygon([t0+2*LW2,l0+2*LW2,
                                       t0+VW-2*LW2,l0+2*LW2,
                                       t0+VW-2*LW2,l0+HW-2*LW2,
                                       t0+2*LW2,l0+HW-2*LW2], 
                                       outline=SHAPECOLOR, fill=BGCOLOR, width=LW1, smooth=True)
                
    def DrawSelection(self, canvas):
        for (i,j) in self.selected:
            t=VP+VW*i
            l=HP+HW*j
            b=VP+VW*(i+1)
            r=HP+HW*(j+1)
            canvas.create_rectangle(t, l, b, r, fill=SELCOLOR, outline='', stipple=SELSTIPPLE)

    def DrawState(self, canvas, grid):
        for i in range(9):
            for j in range(9):
                digit = grid.get_digit(i,j)
                cpm = grid.get_centerpm(i,j)
                allcpm=''
                if cpm=='' and self.allcpm:
                    allcpm = grid.get_centerpm(i,j, forced=True)
                tpm = grid.get_toppm(i,j)
                if digit!=0:       
                    color = FONTCOLOR
                    if grid.is_orig(i,j):
                        color = ORIGFONTCOLOR
                    canvas.create_text(VP+VW*i+VW/2,HP+HW*j+HW/2,
                                       anchor = "center",
                                       fill=color,
                                       font=FONT+" "+FONTSIZE,
                                       text=str(digit))
                else:
                    if cpm!='':                
                        canvas.create_text(VP+VW*i+VW/2-PM_ADJ,HP+HW*j+HW/2,
                                           anchor = "center",
                                           fill=FONTCOLOR,
                                           font=FONT_PM+" "+FONTSIZE_PM,
                                           text=cpm)
                    elif allcpm!='':
                        canvas.create_text(VP+VW*i+VW/2-PM_ADJ,HP+HW*j+HW/2,
                                           anchor = "center",
                                           fill=INACTIVECPMCOLOR,
                                           font=FONT_PM+" "+FONTSIZE_PM,
                                           text=allcpm)                                                
                    if tpm!='':
                        canvas.create_text(VP+VW*i+VW/2-PM_ADJ,HP+HW*j+HW/2-TOPPM_PADDING,
                                           anchor = "center",
                                           fill=FONTCOLOR,
                                           font=FONT_PM+" "+FONTSIZE_PM+ " italic",
                                           text=tpm)
    def DrawFrame(self, canvas, i, j):
        if self.framelines:
            for e in self.framelines:
                canvas.delete(e)
                
        e = canvas.create_rectangle( VP+VW*i, HP+HW*j, 
                                     VP+VW*(i+1), HP+HW*(j+1), 
                                     outline=FRAMECOLOR, width=FRAMEWIDTH)
        self.framelines.append(e)
            
    def Draw(self, delete=True):
        if delete:
            self.canvas.delete('all')
        grid = self.gridlist.getcurr()
        self.DrawColors(self.canvas, grid)
        self.DrawSelection(self.canvas)        
        self.DrawBase(self.canvas)
        self.DrawShapes(self.canvas, grid)
        self.DrawState(self.canvas, grid)
        self.DrawFrame(self.canvas, self.curri, self.currj)

        
    ##########################
    # Event handler methods    
    
    def key_arrow(self, event):
        oldi = self.curri
        oldj = self.currj
        if event.keycode==38:    #up
            self.currj-=1+9
            self.currj%=9
        elif event.keycode==40:  #down
            self.currj+=1
            self.currj%=9
        elif event.keycode==37:  #left
            self.curri-=1+9
            self.curri%=9
        elif event.keycode==39:  #right
            self.curri+=1
            self.curri%=9
        if event.state & 0x0001: #shift
            if not (oldi,oldj) in self.selected:
                self.selected.add( (oldi,oldj) )
            if not (self.curri,self.currj) in self.selected:
                self.selected.add( (self.curri,self.currj) )
            self.Draw()
        elif event.state & 0x0004: #control
            if (oldi,oldj) in self.selected:
                self.selected.remove( (oldi,oldj) )
            if (self.curri,self.currj) in self.selected:
                self.selected.remove( (self.curri,self.currj) )
            self.Draw()
        else:
            self.DrawFrame(self.canvas, self.curri, self.currj)


    def key(self, event):
        # Numbers from 1 to 9
        if event.keycode>=49 and event.keycode<=57:            
            digit = event.keycode - 48
            if event.state & 0x0004: #control
                self.gridlist.change_toppm(self.selected, self.curri, self.currj, digit)
            elif event.state & 0x0001: #shift
                self.gridlist.change_centerpm(self.selected, self.curri, self.currj, digit)
            elif event.state & 0x20000: #alt
                self.gridlist.add_base_digit(self.selected, self.curri, self.currj, digit)
            else: #no modifier
                self.gridlist.add_digit(self.selected, self.curri, self.currj, digit)            
        # Selecting current cell
        elif event.char==' ':
            if (self.curri,self.currj) in self.selected:
                self.selected.remove( (self.curri,self.currj) )
            else:
                self.selected.add( (self.curri,self.currj) )
        self.Draw()
        
    def undo(self, event):
        self.gridlist.undo()
        self.Draw()
        
    def redo(self, event):
        self.gridlist.redo()
        self.Draw()

    def undo_base(self, event):
        self.gridlist.undo_base()
        self.Draw()
        
    def redo_base(self, event):
        self.gridlist.redo_base()
        self.Draw()
        
        
    def select_all(self, event):
        self.selected = set()
        for i in range(9):
            for j in range(9):
                self.selected.add((i,j))
        self.Draw()

    def deselect_all(self, event):
        self.selected = set()
        self.Draw()
        
    def centerpm_onoff(self, event):
        self.gridlist.centerpm_onoff(self.selected, self.curri, self.currj)
        self.Draw()
    
    def show_all_centerpm(self, event):
        self.allcpm = 1 - self.allcpm
        self.Draw()
        
    def leftclick(self, event):
        x = event.x-HP
        y = event.y-VP
        i = x//HW
        j = y//VW
        if i>=0 and i<9 and j>=0 and j<9:
            self.curri = i
            self.currj = j
            if event.state & 0x0004: #control
                if (self.curri,self.currj) in self.selected:
                    self.selected.remove( (self.curri,self.currj) )
                self.Draw()                                            
            elif event.state & 0x0001: #shift
                if not (self.curri,self.currj) in self.selected:
                    self.selected.add( (self.curri,self.currj) )            
                self.Draw()                            
            else: # neither control nor shift
                self.DrawFrame(self.canvas, self.curri, self.currj)            
    
    def inc_showlevel(self, event):
        self.gridlist.change_showlevel(1)
        self.Draw()
    
    def dec_showlevel(self, event):
        self.gridlist.change_showlevel(-1)
        self.Draw()
        
    def set_color(self, color):
        self.gridlist.change_color(self.selected, self.curri, self.currj, self.currcolor)
        self.Draw()
    
    def inc_currcolor(self, event):
        self.currcolor = (self.currcolor + 1) % 10 #Colors go from 0 to 9
        self.Draw()

    def dec_currcolor(self, event):
        self.currcolor = (self.currcolor - 1) % 10 #Colors go from 0 to 9
        self.Draw()
        
    def next_shape_base(self, event):
        if not self.selected:
            self.gridlist.next_1cellshape_base(self.curri, self.currj)
        elif len(self.selected)==2:
            l = list(self.selected)
            l.sort()
            i1=l[0][0]
            j1=l[0][1]
            i2=l[1][0]
            j2=l[1][1]
            self.gridlist.next_2cellshape_base(i1,j1,i2,j2)
        elif len(self.selected)==4:
            pass
        self.Draw()
        
    def next_line_segment_base(self, event):
        if len(self.selected)==2:
            l = list(self.selected)
            l.sort()
            i1=l[0][0]
            j1=l[0][1]
            i2=l[1][0]
            j2=l[1][1]
            self.gridlist.next_line_segment_base(i1,j1,i2,j2)
            self.Draw()

        

In [75]:
ui = UI()
ui.mainloop()